# Get all items by location and update the Call Number Type

This notebook fetches all items in a specified location and then updates the Call Number type field in the Folio Holdings record.


## Preparations

### Install modules
Most modules that we will use are already installed on Google Colab. The below we need to actively install.

In [ ]:
# We will use folioclient to communicate with FOLIO: https://pypi.org/project/folioclient/

!pip install folioclient

In [ ]:
# We will use xlsxwriter to create Excel files: https://pypi.org/project/XlsxWriter/

!pip install xlsxwriter

### Import modules
Modules that are installed need to be imported, so that we can access them in our code.

In [ ]:
import requests
import pandas
from pandas import json_normalize
from folioclient import FolioClient
from getpass import getpass
from google.colab import files

## Connect to FOLIO
We log into FOLIO by providing the okapi URL and the tenant ID, as well as the username and password of a FOLIO user. FOLIO will return a token which contains information about which user we are acting as, and what permissions that user has.

Note that tokens and passwords are sensitive data which should always be kept secret and secure.

### Provide tenant information and credentials

In [ ]:
okapi_url = "YOUR URL HERE"
tenant_id = "YOUR TENANT ID HERE"

# The getpass module lets us enter credentials at runtime
username = getpass('Enter username here: ')
password = getpass('Enter password here: ')

### Log into FOLIO
We will use the FolioClient module to make requests to FOLIO.

In [ ]:
# This will log us into FOLIO, and temporarily store important information like the token in the background
try:
  folio_client = FolioClient(okapi_url, tenant_id, username, password)
  print(f"Successfully logged into FOLIO as user {username}")
except Exception as e:
  print(e)

## Design the search query
This is the part where you specify what data you want to get from FOLIO.

### Endpoint, object and page size
Specify the endpoint you want to search, what objects you expect to get back, and an appropriate page size.

Appropriate page size varies by endpoint. For /inventory/items the maximum is around 100, but for /item-storage/items you can get tens of thousands of records at a time.

# Full list of UUIDs in FOLIO Locations and Service Points

In [ ]:
endpoint = "/holdings-storage/holdings"
record_type = "holdingsRecords"
page_size = "1000"

#query = '?query=cql.AllRecords=1 NOT callNumberTypeId="" AND callNumber="00*" AND callNumber="09*"' #Dewey query
query = '?query=cql.AllRecords=1 NOT callNumberTypeId="" AND permanentLocationId=="36c5155d-a5e4-41e9-b195-ab53099c491b"'
#query = '?query=cql.AllRecords=1 NOT callNumberTypeId="" AND permanentLocationId==36c5155d-a5e4-41e9-b195-ab53099c491b AND callNumber=="TR*"' #LC query
#query = '?query=cql.AllRecords=1 NOT callNumberTypeId="" AND permanentLocationId==d87be268-d156-4219-85be-c848f720f6a6  AND item.materialTypeId==62c70b15-f1fc-41af-abd3-b38d93af' #use an item type to narrow down results
#query = '?query=cql.AllRecords=1 NOT callNumberTypeId="" AND callNumber>="090*" AND callNumber<="099*"' #Dewey query
#query = '?query=cql.AllRecords=1 AND permanentLocationId==e46ce021-3fc7-43c8-acfc-26dfcb2b2a3e AND callNumber>="CD-99999999"' #oversize cd query using a local call number scheme

### Select locations to work with
One way to do this is to search by code/name, or recurring patterns or prefixes in the code/name.

In [ ]:
# Get all the locations you have in FOLIO
all_locations = pandas.json_normalize(folio_client.locations)[["code", "name", "id"]]
print(f"Found {len(all_locations)} locations in total.")

In [ ]:
# Specify a location code prefix which identifies the locations you want to fetch
location_code_prefix = "YOUR PREFIX HERE"

In [ ]:
# Select the locations where the code starts with the code prefix
selected_locations = all_locations.loc[all_locations["code"].str.startswith(location_code_prefix)]
selected_locations = selected_locations.to_dict("records")
print(f"Found {len(selected_locations)} locations where the code starts with the pattern {location_code_prefix}")

## Get the item data from FOLIO
This is where you actually fetch the data from FOLIO.

If your question is quite simple, and only requires you to interact with one API, you can do this with one query as in the example below.
* What are the titles and barcodes of all the items with this effective location?

An example of a more compelx question, which requires queries to multiple APIs, could be
* What are the names of all users who have ever borrowed a book from x location?

### Fetch all the items which match your query

In [ ]:
# Create an empty list. This is the basket where you will store retrieved items.
data = []

# Fetch all the results from FOLIO, in batches of whatever your page size is
for i, record in enumerate(folio_client.folio_get_all(
    path = endpoint, key = record_type, query = query, limit = int(page_size))):

  # Put the results in the basket
  data.append(record)

  # Print out progress
  page_number = int(i / int(page_size))
  if (i > 0) and i % int(page_size) == 0:
      print(f"Fetched {i} records in {page_number} pages.")

print(f"Done! Fetched {i+1} records in {page_number + 1} pages.\n")

## Work with the data
There is a lot you can do here: create different tables based on various critera, analyse the content of the data, rename headers, change the content of the data.

### Read the item data into a dataframe and inspect it
A dataframe is a table-like structure similar to what you might be used working with in for example Excel.

In [ ]:
  # Read the json data into a dataframe
  df = pandas.json_normalize(data)

  # Inspect the headers in the dataframe
  df.info()
  df.head()

ADD call number type to holding


In [ ]:
updated_holdings = []

for holdings in data:
   holdings["callNumberTypeId"] = "95467209-6d7b-468b-94df-0f5d7ad2747d"
   updated_holdings.append(holdings)

#Call Number Type Cheatsheet
#LC - 95467209-6d7b-468b-94df-0f5d7ad2747d
#Dewey - 03dd64d0-5626-4ecd-8ece-4531e0069f35
#Other Scheme - 6caca63e-5651-4db6-9247-3205156e9699
#Title - 5ba6b62e-6858-490a-8102-5b1369873835

#print(*updated_holdings[:2000], sep="\n") the brackets say how many you will update out of the total; remove to do all; 5000 records is too many, 3000 records is ok
print(*updated_holdings[:3000], sep="\n")

In [ ]:
test_holdings = updated_holdings[:4000]
print(test_holdings)

In [ ]:
successfully_updated = 0


for i, updated_holding in enumerate(test_holdings):
  uuid = updated_holding["id"]
  body = updated_holding
  response = requests.put(f"{okapi_url}/holdings-storage/holdings/{uuid}", json=body, headers=folio_client.okapi_headers)

  if response.status_code == 204:
      successfully_updated += 1
  else:
      print(response.status_code, response.text)

 # Print out progress

  if (i > 0) and i % 500 == 0:
      print(f"Completed {i} records.")

print(f"Successfully updated: {successfully_updated}")

In [ ]:
#Repeat steps (rerun the cells) from the input query to the end until all holdings records have been updated.
#29 February 2024 update: We haven't figured out how to loop the script so it would continue automatically until all the records found matching the query = 0